In [5]:
!pip install ultralytics split-folders

# AIHUB 과일 데이터
## 어노테이션용 모델 학습
- Validset에서 이미지 3600장 층화추출(수동)
- 640x640 리사이즈
- Roboflow로 증강
- 파인튜닝


In [6]:
# 이미지 리사이즈 전처리 
from modules.preprocess_image import resize_and_save_images

base_folder = 'C:\FINAL_PROJECT\DATA\AIhub_Fruit\원본데이터\01.데이터'
resize_and_save_images(base_folder)

In [ ]:
# 어노테이션용 모델 파인튜닝 
## Colab에서 실행 

from ultralytics import YOLO

model = YOLO("models/yolov8m-seg.pt") # fine tuning할 대상 모델파일 경로.
model.train(
    data = r"/content/drive/MyDrive/Data/roboflow_augmented/data.yaml",
    epochs=100,
    patience=25,   # early stopping 설정. (patience epoch 동안 성능 개선이 안되면 학습 조기중단)
    batch=32,       #batch size (256 이하 - 메모리가 허용하는 크기.)
    imgsz=640,    # 입력 이미지 크기 (default: 640 - YOLO가 학습한 이미지의 크기.)
    optimizer="AdamW",
    lr0=0.0005,
    lrf=0.2
    # resume=True # 이어서 학습
)

## 학습 데이터 어노테이션

- Train set 전체 이미지(대봉감 제외) 확인
- 어노테이션 생성 

In [ ]:
# 이미지 확인
import glob
import os

imgdir =r'C:\FINAL_PROJECT\DATA\AIhub_Fruit\images'
imgs=glob.glob(os.path.join(imgdir,'**/*.png'),recursive=True)
len(imgs) # persimmon_daebong 대봉감 삭제 후 78323

78323

In [ ]:
# 어노테이션 생성  및 딕셔너리 저장 
from modules.create_ann import create_ann
from ultralytics import YOLO

imgdir =r'C:\FINAL_PROJECT\DATA\AIhub_Fruit\images'
output_dir = r'C:\FINAL_PROJECT\DATA\AIhub_Fruit\annotations'
ann_model =r"C:\FINAL_PROJECT\models\train\weights\best.pt"
model = YOLO(ann_model)

create_ann(imgdir, output_dir, model,'train',5)

## Train / Valid 분리
```
split/
├── images/
│   ├── train/
│   │   └── .png ...
│   └── val/
│       └── .png ...
└── annotations/
    ├── train/
    │   └── .txt ...
    └── val/
        └── .txt...
```

In [ ]:
import os
from modules.split import split_images_and_annotations

# 원본 디렉토리
input_dir = r'C:\FINAL_PROJECT\DATA\AIhub_Fruit'
output_dir = r'C:\FINAL_PROJECT\DATA\AIhub_Fruit\split'

# 어노테이션 폴더 경로
annotations_folder = os.path.join(input_dir, 'annotations')

# 이미지와 어노테이션 파일을 분할하여 각각의 폴더에 저장
split_images_and_annotations(input_dir, output_dir, annotations_folder)


## 이미지 전처리  
- 랜덤배경 피클 생성 및 저장
- 과일 + 랜덤배경 합성

In [ ]:
# 랜덤 배경 피클 생성 및 저장 
from modules.random_background import generate_random_backgrounds

img_dir = r"C:\classes\FINAL_PROJECT\Data\background\from_coco_test" # 400장 추출 
bgr_pkl_path = r"C:\classes\FINAL_PROJECT\output"
generate_random_backgrounds(img_dir, bgr_pkl_path, num_backgrounds=1000, target_size= 640)

In [ ]:
# 과일 랜덤 배경 합성
from modules.add_background import add_background

img_dir = r"C:\FINAL_PROJECT\DATA\AIhub_Fruit\split\images"
output_dir = r"C:\FINAL_PROJECT\DATA\AIhub_Fruit\with_background"
bgr_pkl_path = r"output/random_background.pkl"

add_background(img_dir, output_dir, bgr_pkl_path)


# 기타
## 손 데이터
- 크롭, 리사이즈(640x640)


In [ ]:
# 손 데이터 정사각형으로 크롭 후 저장
from modules.preprocess_image import crop_and_save_images

img_dir = r"C:\classes\FINAL_PROJECT\Data\AIHUB_hand"
output_dir = r"C:\classes\FINAL_PROJECT\Data\hand_cropped"

crop_and_save_images(img_dir, output_dir)


## 클래스 id 수정 
- Roboflow 증강 후 클래스 id 변경 오류 => 수정
  

In [ ]:
from modules.correct_cls_id import correct_cls_id

## 어노테이션 파일에서 클래스 아이디 수정 
ann_dir = r"C:\classes\FINAL_PROJECT\Data\AIHUB_Fruit\1.Training\Final_Data_Augmented\hand.v5i.yolov8\train\labels"
correct_cls_id(ann_dir)
